In [ ]:
import os
import sys
import numpy as np
import nibabel as nib
import pickle
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir('/local/home/dhaziza/entrack')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
sys.path.append('/local/home/dhaziza/entrack/')

from src.data.providers.py_streaming import get_train_test_filenames

config = {
  'classes': ['healthy', 'health_pd'],
  'data_paths': {
    'datadir': '/local/PPMI/_brain_extracted_1mm/',
    'class_labels': '/local/PPMI/PPMI_T2_NC_PD_100_labels.pkl',
    'valid_data': '/local/PPMI/PPMI_T2_NC_PD_100_test.pkl',
    'train_data': '/local/PPMI/PPMI_T2_NC_PD_100_train.pkl',
    'regex': '\.nii\.gz',
    'split_on': '.nii.gz',
  }
}
train, valid = get_train_test_filenames(config)

In [ ]:
UNIQ_PREFIX = 'pd_brain_extracted_1mm'
ROOT_DATA_DIR = 'notebooks/2d_crops_%s' % UNIQ_PREFIX
scale_factor = 2
z_slices = range(scale_factor*18, scale_factor*72, scale_factor*1)

def normalize(v):
    v = v - np.mean(v)
    norm = np.std(v)
    if norm == 0:
        return v
    return v / norm

def dump_files(files, _class, train, plot=False):
    total_todo = len(files)
    print_every = 300
    train_text = 'train' if train else 'valid'
    files.sort()
    for i, f in enumerate(files):
        data = nib.load(f).get_data().astype(np.float32)
        data = normalize(data)
        for z in z_slices:
            try:
                os.makedirs('%s/z%d/%s_c%d_z%d' % (ROOT_DATA_DIR, z, UNIQ_PREFIX, _class, z))
            except OSError: # Directory exists
                pass
            save_to = '%s/z%d/%s_c%d_z%d/%d_%s.jpg' % (
                ROOT_DATA_DIR, z, UNIQ_PREFIX, _class, z, i, train_text)
            sliced_data = data[:, :, z]
            sliced_data = (sliced_data*10 + 128)
            sliced_data = sliced_data.astype(np.uint8)
            im = Image.fromarray(sliced_data)
            im.save(save_to)
        if i % print_every == 0:
            print('c%d[%s]: Done %d / %d' % (_class, train_text, i, total_todo))
        
dump_files(train[0], 0, True)
dump_files(train[1], 1, True)
dump_files(valid[0], 0, False)
dump_files(valid[1], 1, False)